<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglés.

In [5]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip   
else:
    print("El dataset ya se encuentra descargado")

--2022-08-03 11:45:15--  http://songs_dataset.zip/
Resolving songs_dataset.zip (songs_dataset.zip)... failed: Name or service not known.
wget: unable to resolve host address ‘songs_dataset.zip’
--2022-08-03 11:45:15--  https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip [following]
--2022-08-03 11:45:15--  https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.

In [6]:
# Posibles bandas
os.listdir("./songs_dataset/")

['radiohead.txt',
 'Lil_Wayne.txt',
 'kanye.txt',
 'janisjoplin.txt',
 'amy-winehouse.txt',
 'prince.txt',
 'nirvana.txt',
 'ludacris.txt',
 'blink-182.txt',
 'rihanna.txt',
 'bieber.txt',
 'leonard-cohen.txt',
 'lorde.txt',
 'r-kelly.txt',
 'patti-smith.txt',
 'dr-seuss.txt',
 'jimi-hendrix.txt',
 'britney-spears.txt',
 'adele.txt',
 'nicki-minaj.txt',
 'lil-wayne.txt',
 'dolly-parton.txt',
 'bruno-mars.txt',
 'bob-dylan.txt',
 'al-green.txt',
 'joni-mitchell.txt',
 'michael-jackson.txt',
 'notorious-big.txt',
 'eminem.txt',
 'alicia-keys.txt',
 'cake.txt',
 'bob-marley.txt',
 'dickinson.txt',
 'nickelback.txt',
 'beatles.txt',
 'dj-khaled.txt',
 'missy-elliott.txt',
 'notorious_big.txt',
 'kanye-west.txt',
 'disney.txt',
 'bjork.txt',
 'lin-manuel-miranda.txt',
 'Kanye_West.txt',
 'bruce-springsteen.txt',
 'nursery_rhymes.txt',
 'drake.txt',
 'paul-simon.txt',
 'lady-gaga.txt',
 'johnny-cash.txt']

In [7]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/lady-gaga.txt', sep='/n', header=None)
df.head()

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,0
0,"I'll undress you, 'cause you're tired"
1,Cover you as you desire
2,When you fall asleep inside my arms
3,May not have the fancy things
4,But I'll give you everything


In [8]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 3807


### 1 - Preprocesamiento

In [9]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [10]:
# Demos un vistazo
sentence_tokens[:2]

[["i'll", 'undress', 'you', "'cause", "you're", 'tired'],
 ['cover', 'you', 'as', 'you', 'desire']]

### 2 - Crear los vectores (word2vec)

In [11]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobracargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [12]:
# Crearmos el modelo generador de vectoeres
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=3,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [13]:
# Buildear el vocabularui con los tokens
w2v_model.build_vocab(sentence_tokens)

In [14]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 3807


In [15]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.vocab))

Cantidad de words distintas en el corpus: 722


### 3 - Entrenar el modelo generador

In [17]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=60,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 32898.37109375
Loss after epoch 1: 31340.21484375
Loss after epoch 2: 31276.625
Loss after epoch 3: 31092.078125
Loss after epoch 4: 31112.7421875
Loss after epoch 5: 30202.890625
Loss after epoch 6: 29923.25
Loss after epoch 7: 29202.828125
Loss after epoch 8: 29015.875
Loss after epoch 9: 27638.75
Loss after epoch 10: 28600.375
Loss after epoch 11: 27963.15625
Loss after epoch 12: 28095.25
Loss after epoch 13: 26104.4375
Loss after epoch 14: 27173.40625
Loss after epoch 15: 25577.125
Loss after epoch 16: 25511.28125
Loss after epoch 17: 27100.21875
Loss after epoch 18: 26405.75
Loss after epoch 19: 25676.4375
Loss after epoch 20: 24868.3125
Loss after epoch 21: 25411.8125
Loss after epoch 22: 25332.5625
Loss after epoch 23: 25200.4375
Loss after epoch 24: 25527.3125
Loss after epoch 25: 24885.5625
Loss after epoch 26: 24902.25
Loss after epoch 27: 24399.75
Loss after epoch 28: 24425.0
Loss after epoch 29: 24984.5
Loss after epoch 30: 24920.25
Loss after epoch 31: 

(1116660, 1818000)

### 4 - Ensayar

In [33]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["queen"], topn=10)

[('holy', 0.6926374435424805),
 ('dry', 0.6195493936538696),
 ('cured', 0.5951551198959351),
 ('sweet', 0.5935732126235962),
 ('austa', 0.5763121247291565),
 ('gypsy', 0.5675322413444519),
 ('strong', 0.5672372579574585),
 ('schleiban', 0.5626291632652283),
 ('caught', 0.5581063032150269),
 ('pig', 0.5503557324409485)]

In [20]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["way"], topn=10)

[('inside', 0.12464039772748947),
 ('take', 0.008284851908683777),
 ('myself', 0.001946520060300827),
 ('hurt', -0.0031950436532497406),
 ('fall', -0.01628391072154045),
 ("we're", -0.017314940690994263),
 ('gold', -0.05167284607887268),
 ('ya', -0.055799245834350586),
 ('been', -0.05584544688463211),
 ('pig', -0.061395756900310516)]

In [23]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["alejandro"], topn=10)

[('roberto', 0.82742840051651),
 ('ale', 0.7167729139328003),
 ('fernando', 0.6841555833816528),
 ('jandro', 0.6814889907836914),
 ('knows', 0.5864957571029663),
 ('call', 0.5813551545143127),
 ('dont', 0.5735774636268616),
 ('giving', 0.5700219869613647),
 ('reach', 0.5626158714294434),
 ('lose', 0.5446866750717163)]

In [24]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["telephone"], topn=5)

[('reach', 0.9054993391036987),
 ('m', 0.6676629781723022),
 ('roberto', 0.6357553005218506),
 ('marry', 0.6191132664680481),
 ('sorry', 0.5938770174980164)]

In [35]:
# Ensayar con una palabra que no está en el corpus (en vocab):
w2v_model.wv.most_similar(negative=["Nebraska"])

KeyError: ignored

### 5 - Visualizar agrupación de vectores

In [36]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index2word)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [38]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.

